In [ ]:
import nibabel as nib
import numpy as np
import scipy
import scipy.io as sio
import pandas as pd

# for upsampling
from slicetime.main import run_slicetime
from slicetime.make_image_stack import make_image_stack

import os
import os.path as op
import time
import urllib.request
from tqdm import tqdm # progress bars
from pprint import pprint
import warnings
warnings.filterwarnings('ignore')

from glmsingle.glmsingle import GLM_single

In [ ]:
base_folder = op.join('D:\\', 'Zsuzsa', 'HCCCL', 'miniTRK', 'Results')
mri_folder = op.join(base_folder, '01_MRI', 'fMRI_Preproc', 'VER3', 'aligned')
design_folder = op.join(base_folder, '02_APS_MRI_Logs', 'single_trials')
#subjects = np.loadtxt('test_subjects.txt', dtype=str)
subjects = ['991905']
print(subjects)

In [ ]:
opt = dict()

# set important fields for completeness (but these would be enabled by default)
opt['wantlibrary'] = 0
opt['wantglmdenoise'] = 0
opt['wantfracridge'] = 0

# for the purpose of this example we will keep the relevant outputs in memory
# and also save them to the disk
opt['wantfileoutputs'] = [1,0,0,0]
opt['wantmemoryoutputs'] = [1,0,0,0]

# running python GLMsingle involves creating a GLM_single object
# and then running the procedure using the .fit() routine
glmsingle_obj = GLM_single(opt)

# visualize all the hyperparameters
pprint(glmsingle_obj.params)

# OBJECT ENCODING

In [ ]:
task = 'OBJ'
acq = 'ENC'
stimdur = 3.0
tr = 1

## Upsample Data

In [ ]:
tr_old = 1.84
tr_new = tr
time_dim = 3
offset = 0
n_slices = 80

for subj in subjects:
    
    for r in ['1','2']:
        start_time = time.time()
        
        original_fname = subj + '_' + task + '_' + acq + '_' + r + '_To_ObjEnc1MeanFunc.nii.gz'
        upsampled_fname = subj + '_' + task + '_' + acq + '_' + r + '_To_ObjEnc1MeanFunc_Upsampled.nii.gz'
        in_file = op.join(mri_folder, original_fname)
        out_file = op.join(mri_folder, upsampled_fname)

        # sliceorder needs to be 1-based (see fakeout below)
        slicetimes = np.flip(np.arange(0, tr_old, tr_old/n_slices))
        
        run_slicetime(
        inpath=in_file,
        outpath=out_file,
        slicetimes=slicetimes,
        tr_old=tr_old,
        tr_new=tr_new,
        )
        
        elapsed_time = time.time() - start_time

        print(
            '\telapsed time: ',
            f'{time.strftime("%H:%M:%S", time.gmtime(elapsed_time))}'
        )

## Run GLMsingle

In [ ]:
for subj in subjects:
    
    # load runs and design files
    data = []
    designs = []
    for r in ['1','2']:
        nii_file = subj + '_' + task + '_' + acq + '_' + r + '_To_ObjEnc1MeanFunc.nii.gz'
        fname = op.join(mri_folder, nii_file)
        img = nib.load(fname)
        data.append(img.get_fdata())
        
        design_file =  subj + '_' + task + '_SingleTrials_run_' + r + '.csv' 
        fname = op.join(design_folder, design_file)
        design = pd.read_csv(fname).to_numpy()
        designs.append(design)
            
    # create a directory for saving GLMsingle outputs
    outputdir_glmsingle = op.join(base_folder,'01_MRI','fMRI_RSA','GLMsingle', subj, task + '_' + acq)
    figuredir_glmsingle = op.join(outputdir_glmsingle, 'figures')
    
    print(subj)
    start_time = time.time()
    print(start_time)

    if not op.exists(outputdir_glmsingle):

        print(f'running GLMsingle...')

        # run GLMsingle
        results_glmsingle = glmsingle_obj.fit(
           designs,
           data,
           stimdur,
           tr,
           outputdir=outputdir_glmsingle,
           figuredir=figuredir_glmsingle
        )

    else:
        print(f'loading existing GLMsingle outputs from directory:\n\t{outputdir_glmsingle}')

        # load existing file outputs if they exist
        results_glmsingle = dict()
        results_glmsingle['typea'] = np.load(op.join(outputdir_glmsingle,'TYPEA_ONOFF.npy'),allow_pickle=True).item()
        results_glmsingle['typeb'] = np.load(op.join(outputdir_glmsingle,'TYPEB_FITHRF.npy'),allow_pickle=True).item()
        results_glmsingle['typec'] = np.load(op.join(outputdir_glmsingle,'TYPEC_FITHRF_GLMDENOISE.npy'),allow_pickle=True).item()
        results_glmsingle['typed'] = np.load(op.join(outputdir_glmsingle,'TYPED_FITHRF_GLMDENOISE_RR.npy'),allow_pickle=True).item()

    elapsed_time = time.time() - start_time

    print(
        '\telapsed time: ',
        f'{time.strftime("%H:%M:%S", time.gmtime(elapsed_time))}'
    )
    

In [ ]:
del results_glmsingle